In [1]:
#import libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.metrics import confusion_matrix,auc
import warnings
warnings.filterwarnings("ignore")


In [2]:
#import dataset

telco_dataset = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
telco_dataset.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [11]:
print("cols and rows in the dataset", telco_dataset.shape," \n")
print("missing values in the dataset \n\n",telco_dataset.isnull().sum())
print("\n\n data types \n",telco_dataset.dtypes)

cols and rows in the dataset (7043, 21)  

missing values in the dataset 

 customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


 data types 
 customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract     

In [39]:
telco_dataset['TotalCharges'] = telco_dataset["TotalCharges"].replace(" ",np.nan)

#Dropping null values from total charges column which contain .15% missing data 
telco_dataset = telco_dataset[telco_dataset["TotalCharges"].notnull()]
telco_dataset = telco_dataset.reset_index()[telco_dataset.columns]

telco_dataset["TotalCharges"] = telco_dataset["TotalCharges"].astype(float)

replace_cols = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport','StreamingTV', 'StreamingMovies']
for i in replace_cols : 
    telco_dataset[i]  = telco_dataset[i].replace({'No internet service' : 'No'})
    
#replace values
telco_dataset["SeniorCitizen"] = telco_dataset["SeniorCitizen"].replace({1:"Yes",0:"No"})

#Tenure to categorical column
def tenure_lab(telco_dataset):
    
    if telco_dataset["tenure"] <= 12 :
        return "Tenure_0-12"
    elif (telco_dataset["tenure"] > 12) & (telco_dataset["tenure"] <= 24 ):
        return "Tenure_12-24"
    elif (telco_dataset["tenure"] > 24) & (telco_dataset["tenure"] <= 48) :
        return "Tenure_24-48"
    elif (telco_dataset["tenure"] > 48) & (telco_dataset["tenure"] <= 60) :
        return "Tenure_48-60"
    elif telco_dataset["tenure"] > 60 :
        return "Tenure_gt_60"
telco_dataset["tenure_group"] = telco_dataset.apply(lambda telco_dataset:tenure_lab(telco_dataset),
                                      axis = 1)

#Separating churn and non churn customers
churn     = telco_dataset[telco_dataset["Churn"] == "Yes"]
not_churn = telco_dataset[telco_dataset["Churn"] == "No"]


TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

In [40]:
#Separating catagorical and numerical columns
Id_col     = ['customerID']
target_col = ["Churn"]
cat_cols   = telco_dataset.nunique()[telco_dataset.nunique() < 6].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]
num_cols   = [x for x in telco_dataset.columns if x not in cat_cols + target_col + Id_col]

In [41]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

#customer id col
Id_col     = ['customerID']
#Target columns
target_col = ["Churn"]
#categorical columns
cat_cols   = telco_dataset.nunique()[telco_dataset.nunique() < 6].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]
#numerical columns
num_cols   = [x for x in telco_dataset.columns if x not in cat_cols + target_col + Id_col]
#Binary columns with 2 values
bin_cols   = telco_dataset.nunique()[telco_dataset.nunique() == 2].keys().tolist()
#Columns more than 2 values
multi_cols = [i for i in cat_cols if i not in bin_cols]

#Label encoding Binary columns
le = LabelEncoder()
for i in bin_cols :
    telco_dataset[i] = le.fit_transform(telco_dataset[i])
    
#Duplicating columns for multi value columns
telco_dataset = pd.get_dummies(data = telco_dataset,columns = multi_cols )

#Scaling Numerical columns
std = StandardScaler()
scaled = std.fit_transform(telco_dataset[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)

#dropping original values merging scaled values for numerical columns
df_telcom_og = telco_dataset.copy()
telco_dataset = telco_dataset.drop(columns = num_cols,axis = 1)
telco_dataset = telco_dataset.merge(scaled,left_index=True,right_index=True,how = "left")

In [47]:
##copy data
#telco_df = telco_dataset.copy()
#Drop tenure column
correlation = telco_dataset.corr()
correlation
#telco_dataset = telco_dataset.drop(columns = "tenure_group",axis = 1)

,gender,SeniorCitizen,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,...,PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_Tenure_0-12,tenure_group_Tenure_12-24,tenure_group_Tenure_24-48,tenure_group_Tenure_48-60,tenure_group_Tenure_gt_60,tenure,MonthlyCharges,TotalCharges
gender,1.000000,-0.001819,-0.001379,0.010349,-0.007515,-0.016328,-0.013093,-0.000807,-0.008507,-0.007124,...,0.000844,0.013199,-0.001050,-0.000649,-0.010516,-0.004319,0.016279,0.005285,-0.013779,0.000048
SeniorCitizen,-0.001819,1.000000,0.016957,-0.210550,0.008392,-0.038576,0.066663,0.059514,-0.060577,0.105445,...,0.171322,-0.152987,-0.027713,0.001860,0.020383,0.014186,-0.002407,0.015683,0.219874,0.102411
Partner,-0.001379,0.016957,1.000000,0.452269,0.018397,0.143346,0.141849,0.153556,0.120206,0.124483,...,-0.083207,-0.096948,-0.305061,-0.048481,0.028468,0.105341,0.280353,0.381912,0.097825,0.319072
Dependents,0.010349,-0.210550,0.452269,1.000000,-0.001078,0.080786,0.023639,0.013900,0.063053,-0.016499,...,-0.149274,0.056448,-0.145379,-0.001459,0.024645,0.031419,0.118090,0.163386,-0.112343,0.064653
PhoneService,-0.007515,0.008392,0.018397,-0.001078,1.000000,-0.091676,-0.052133,-0.070076,-0.095138,-0.021383,...,0.002747,-0.004463,-0.006950,0.012307,-0.014778,-0.009750,0.020516,0.007877,0.248033,0.113008
OnlineSecurity,-0.016328,-0.038576,0.143346,0.080786,-0.091676,1.000000,0.283285,0.274875,0.354458,0.175514,...,-0.112295,-0.079918,-0.242409,-0.055662,0.007695,0.071663,0.263228,0.328297,0.296447,0.412619
OnlineBackup,-0.013093,0.066663,0.141849,0.023639,-0.052133,0.283285,1.000000,0.303058,0.293705,0.281601,...,-0.000364,-0.174075,-0.267366,-0.084081,0.023085,0.099192,0.278788,0.361138,0.441529,0.510100
DeviceProtection,-0.000807,0.059514,0.153556,0.013900,-0.070076,0.274875,0.303058,1.000000,0.332850,0.389924,...,-0.003308,-0.187325,-0.273920,-0.077332,0.044972,0.076854,0.275537,0.361520,0.482607,0.522881
TechSupport,-0.008507,-0.060577,0.120206,0.063053,-0.095138,0.354458,0.293705,0.332850,1.000000,0.277549,...,-0.114807,-0.084631,-0.238628,-0.072019,0.022137,0.062710,0.265396,0.325288,0.338301,0.432868
StreamingTV,-0.007124,0.105445,0.124483,-0.016499,-0.021383,0.175514,0.281601,0.389924,0.277549,1.000000,...,0.144747,-0.247712,-0.220761,-0.050234,0.028829,0.087969,0.198150,0.280264,0.629668,0.515709


In [43]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from sklearn.metrics import f1_score



In [44]:
train,test = train_test_split(telco_dataset,test_size = .25 ,random_state = 111)
    
##seperating dependent and independent variables
cols    = [i for i in telco_dataset.columns if i not in Id_col + target_col]
train_X = train[cols]
train_Y = train[target_col]
test_X  = test[cols]
test_Y  = test[target_col]


In [46]:
logit  = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

#telecom_churn_prediction(logit,train_X,test_X,train_Y,test_Y,
#                        cols,"coefficients",threshold_plot = True)

logit.fit(train_X,train_Y)
predictions   = logit.predict(test_X)
probabilities = logit.predict_proba(test_X)

    
print(logit)
print("\n Classification report : \n",classification_report(test_Y,predictions))
print("Accuracy   Score : ",accuracy_score(test_Y,predictions))
#confusion matrix
conf_matrix = confusion_matrix(test_Y,predictions)
#roc_auc_score
model_roc_auc = roc_auc_score(test_Y,predictions) 
print("Area under curve : ",model_roc_auc,"\n")
fpr,tpr,thresholds = roc_curve(test_Y,probabilities[:,1])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

 Classification report : 
              precision    recall  f1-score   support

          0       0.83      0.91      0.87      1268
          1       0.69      0.53      0.60       490

avg / total       0.79      0.80      0.79      1758

Accuracy   Score :  0.8026166097838453
Area under curve :  0.7185443893645785 



In [52]:
from sklearn.feature_selection import RFE

rfe = RFE(logit,10)
rfe.fit(train_X,train_Y.values.ravel())

predictions   = rfe.predict(test_X)
probabilities = rfe.predict_proba(test_X)

    
print(rfe)
print("\n Classification report : \n",classification_report(test_Y,predictions))
print("Accuracy   Score : ",accuracy_score(test_Y,predictions))
#confusion matrix
conf_matrix = confusion_matrix(test_Y,predictions)
#roc_auc_score
model_roc_auc = roc_auc_score(test_Y,predictions) 
print("Area under curve : ",model_roc_auc,"\n")
fpr,tpr,thresholds = roc_curve(test_Y,probabilities[:,1])

RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
  n_features_to_select=10, step=1, verbose=0)

 Classification report : 
              precision    recall  f1-score   support

          0       0.82      0.91      0.87      1268
          1       0.69      0.49      0.57       490

avg / total       0.78      0.79      0.78      1758

Accuracy   Score :  0.7946530147895335
Area under curve :  0.6998760703019378 



In [56]:
from sklearn.ensemble import RandomForestClassifier

#function attributes
#columns  - column used
#nf_estimators   - The number of trees in the forest.
#estimated_tree  - tree number to be displayed
#maximum_depth   - depth of the tree
#criterion_type  - split criterion type ["gini" or "entropy"]
#Model performance - prints performance of model

rfc   = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = "gini")
rfc.fit(train_X,train_Y)

predictions   = rfc.predict(test_X)
probabilities = rfc.predict_proba(test_X)
#coeffs
    
print(rfc)
print("\n Classification report : \n",classification_report(test_Y,predictions))
print("Accuracy   Score : ",accuracy_score(test_Y,predictions))
#confusion matrix
conf_matrix = confusion_matrix(test_Y,predictions)
#roc_auc_score
model_roc_auc = roc_auc_score(test_Y,predictions) 
print("Area under curve : ",model_roc_auc,"\n")
fpr,tpr,thresholds = roc_curve(test_Y,probabilities[:,1])


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

 Classification report : 
              precision    recall  f1-score   support

          0       0.80      0.94      0.86      1268
          1       0.71      0.38      0.50       490

avg / total       0.77      0.78      0.76      1758

Accuracy   Score :  0.7849829351535836
Area under curve :  0.6618682804352025 



In [62]:
from sklearn.svm import SVC

#Support vector classifier
#using linear hyper plane
svc_lin  = SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
               decision_function_shape='ovr', degree=3, gamma=1.0, kernel='linear',
               max_iter=-1, probability=True, random_state=None, shrinking=True,
               tol=0.001, verbose=False)
svc_lin.fit(train_X,train_Y)

predictions   = svc_lin.predict(test_X)
probabilities = svc_lin.predict_proba(test_X)
#coeffs
    
print(svc_lin)
print("\n Classification report : \n",classification_report(test_Y,predictions))
print("Accuracy   Score : ",accuracy_score(test_Y,predictions))
#confusion matrix
conf_matrix = confusion_matrix(test_Y,predictions)
#roc_auc_score
model_roc_auc = roc_auc_score(test_Y,predictions) 
print("Area under curve : ",model_roc_auc,"\n")
fpr,tpr,thresholds = roc_curve(test_Y,probabilities[:,1])


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1.0, kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

 Classification report : 
              precision    recall  f1-score   support

          0       0.82      0.92      0.86      1268
          1       0.69      0.47      0.56       490

avg / total       0.78      0.79      0.78      1758

Accuracy   Score :  0.7929465301478953
Area under curve :  0.6924322410352154 



In [60]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB(priors=None)

gnb.fit(train_X,train_Y)

predictions   = gnb.predict(test_X)
probabilities = gnb.predict_proba(test_X)
#coeffs
    
print(gnb)
print("\n Classification report : \n",classification_report(test_Y,predictions))
print("Accuracy   Score : ",accuracy_score(test_Y,predictions))
#confusion matrix
conf_matrix = confusion_matrix(test_Y,predictions)
#roc_auc_score
model_roc_auc = roc_auc_score(test_Y,predictions) 
print("Area under curve : ",model_roc_auc,"\n")
fpr,tpr,thresholds = roc_curve(test_Y,probabilities[:,1])


ImportError: cannot import name '_check_sample_weight' from 'sklearn.utils.validation' (C:\Users\v-ashkha\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py)

In [51]:
from imblearn.over_sampling import SMOTE

cols    = [i for i in telco_dataset.columns if i not in Id_col+target_col]

smote_X = telco_dataset[cols]
smote_Y = telco_dataset[target_col]

#Split train and test data
smote_train_X,smote_test_X,smote_train_Y,smote_test_Y = train_test_split(smote_X,smote_Y,
                                                                         test_size = .25 ,
                                                                         random_state = 111)

#oversampling minority class using smote
os = SMOTE(random_state = 0)
os_smote_X,os_smote_Y = os.fit_sample(smote_train_X,smote_train_Y)
os_smote_X = pd.DataFrame(data = os_smote_X,columns=cols)
os_smote_Y = pd.DataFrame(data = os_smote_Y,columns=target_col)


ImportError: cannot import name 'MultiOutputMixin' from 'sklearn.base' (C:\Users\v-ashkha\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py)